# Computing with NDArray objects

Python-Blosc2 implements a powerful way to operate with NDArray (and other flavors) objects.  In this section, we will see how to do computations with NDArray arrays in a simple way.


In [1]:
import blosc2
import numpy as np

## A simple example
First, let's create a couple of NDArrays.  We will use NumPy arrays to fill them.

In [2]:
shape = (500, 1000)
npa = np.linspace(0, 1, np.prod(shape), dtype=np.float32).reshape(shape)
npb = np.linspace(1, 2, np.prod(shape), dtype=np.float64).reshape(shape)

a = blosc2.asarray(npa, urlpath="a.b2nd", mode="w")
b = blosc2.asarray(npb, urlpath="b.b2nd", mode="w")

Now, let's create an expression that involves `a` and `b`

In [3]:
c = a**2 + b**2 + 2 * a * b + 1
print(c.info)  # at this stage, the expression has not been evaluated yet

type       : LazyExpr
expression : ((((o0 ** 2) + (o1 ** 2)) + ((2 * o0) * o1)) + 1)
operands   : {'o0': 'a.b2nd', 'o1': 'b.b2nd'}
shape      : (500, 1000)
dtype      : float64



We see that the outcome of the expression is a `LazyExpr` object.  This object is a placeholder for the actual computation that will be done when we evaluate it.  This is a very powerful feature because it allows us to build complex expressions without actually computing them until we really need the result.

Now, let's evaluate it. `LazyExpr` objects follow the `LazyArray` interface, and this provides several ways for performing the evaluation, depending on the object we want as the desired output.
 
First, let's use the `eval` method.  The result will be another NDArray array:

In [4]:
d = c.eval()   # evaluate the expression
print(f"Class: {type(d)}")
print(f"Compression ratio: {d.schunk.cratio:.2f}x")

Class: <class 'blosc2.ndarray.NDArray'>
Compression ratio: 1.88x


We can specify different compression parameters for the result.  For example, we can change the codec to `zstd`, use the bitshuffle filter, and the compression level set to 9:

In [5]:
cparams = {
    "codec": blosc2.Codec.ZSTD,
    "filters": [blosc2.Filter.BITSHUFFLE],
    "clevel": 9
}
d = c.eval(cparams=cparams)
print(f"Compression ratio: {d.schunk.cratio:.2f}x")

Compression ratio: 2.03x


Now, let's evaluate the expression and store the result in a NumPy array.  For this, we will use the `__getitem__` method:

In [6]:
npd = d[:]
print(f"Class: {type(npd)}")

Class: <class 'numpy.ndarray'>


## Saving expressions to disk

Finally, you can save expressions to disk.  For this, use the `save` method of ``LazyArray`` objects.  For example, let's save the expression `c` to disk:

In [7]:
c = a**2 + b**2 + 2 * a * b + 1
c.save(urlpath="expr.b2nd")

And you can load it back with the `open` function:

In [8]:
c2 = blosc2.open("expr.b2nd")
print(c2.info)

type       : LazyExpr
expression : ((((o0 ** 2) + (o1 ** 2)) + ((2 * o0) * o1)) + 1)
operands   : {'o0': 'a.b2nd', 'o1': 'b.b2nd'}
shape      : (500, 1000)
dtype      : float64



Now, you can evaluate it as before:

In [9]:
d2 = c2.eval()
print(f"Compression ratio: {d2.schunk.cratio:.2f}x")

Compression ratio: 1.88x


## Broadcasting

NumPy arrays support broadcasting, and so do NDArray arrays.  Let's see an example:


In [10]:
b2 = b[0]  # take the first row of b
print(f"Shape of a: {a.shape}, shape of b2: {b2.shape}")

Shape of a: (500, 1000), shape of b2: (1000,)


We see that the shapes of `a` and `b2` are different.  However, we can still operate with them and the broadcasting will be done automatically (à la NumPy):

In [11]:
c2 = a + b2
d2 = c2.eval()
print(f"Compression ratio: {d2.schunk.cratio:.2f}x, shape: {d2.shape}")

Compression ratio: 18.14x, shape: (500, 1000)


## Reductions

We can also perform reductions on NDArray arrays.  Let's see an example:

In [12]:
c = (a + b).sum()
c

999999.9999999471

As we can see, the result is a scalar. That means that reductions in expressions always perform the computation immediately.  We can also specify the axis for the reduction:

In [13]:
c = (a + b).sum(axis=1)
print(f"Shape of c: {c.shape}")
# Show the first 4 elements of the result
c[:4]

Shape of c: (500,)


array([1001.998004  , 1005.998012  , 1009.99802   , 1013.99802799])

## Summary

In this section, we have seen how to perform computations with NDArray arrays.  We have seen how to create expressions, evaluate them, and save them to disk.  We have also seen how to perform broadcasting and reductions.  This is a very powerful feature that allows you to build and evaluate complex computations from operands that can be in-memory, on-disk or in remote boxes (`C2Array`) in a very simple way.